# 2) Adverbs: Do Great Writers Avoid Them?

**Goal:** Estimate -ly adverb rate and compare across the two texts.

# Setup: Load Texts

This notebook needs **Alice in Wonderland** and **Through the Looking-Glass** as input texts.

**How to provide the texts:**
1. Download books from Project Gutenberg (IDs 11 and 12) as txts. [go to https://www.gutenberg.org/ebooks/11 and https://www.gutenberg.org/ebooks/12]

2. Place two text files in the "data" folder with names:
   - `Wondeland.txt`  (Alice's Adventures in Wonderland)
   - `Looking-Glass.txt` (Through the Looking-Glass)

In [ ]:
import re
from pathlib import Path

In [ ]:

def load_texts(local_alice: str = '../data/Wonderland.txt',
               local_glass: str = '../data/Looking-Glass.txt'):
    """Load Wonderland and Looking-Glass texts from disk.

    Parameters
    ----------
    local_alice : str
        Path to Wonderland text file. Defaults to '../data/Wonderland.txt'.
    local_glass : str
        Path to Looking-Glass text file. Defaults to '../data/Looking-Glass.txt'.

    Returns
    -------
    tuple[str, str]
        (wonderland_text, lookingglass_text).

    Raises
    ------
    FileNotFoundError
        If either file is missing.

    Extra Notes
    -----------
    - Using UTF-8 with `errors='ignore'` avoids codec exceptions on
      older Project Gutenberg dumps or inconsistent encodings.
    """
    p1, p2 = Path(local_alice), Path(local_glass)

    # Fail fast with a clear message if a file is missing
    if not p1.exists():
        raise FileNotFoundError(
            f"Missing file: {p1}\n"
            "→ Please place 'Wonderland.txt' at this path or update load_texts(...)."
        )
    if not p2.exists():
        raise FileNotFoundError(
            f"Missing file: {p2}\n"
            "→ Please place 'Looking-Glass.txt' at this path or update load_texts(...)."
        )

    # Read the files (UTF-8; ignore undecodable bytes to stay robust)
    wonderland   = p1.read_text(encoding='utf-8', errors='ignore')
    lookingglass = p2.read_text(encoding='utf-8', errors='ignore')
    return wonderland, lookingglass

def normalize(text: str) -> str:
    """Normalize a Gutenberg-like text for tokenization.

    Steps
    -----
    1) Heuristically strip Project Gutenberg headers/footers if present
       (looks for *** START ... *** END markers).
    2) Normalize newlines to '\n'.

    Parameters
    ----------
    text : str
        Raw text as loaded from disk (can be empty).

    Returns
    -------
    str
        Cleaned text suitable for tokenization and counting.
    """
    if not text:
        return ''
    # Clip to the main body if markers are present.
    start = text.find('*** START')
    end   = text.find('*** END')
    if start != -1 and end != -1 and end > start:
        text = text[start:end]
    # Normalize Windows line endings.
    return text.replace('\r\n', '\n')

# Load raw texts (forgiving: returns '' if a file is missing)
wonderland_raw, lookingglass_raw = load_texts()

# Normalize for tokenization
wonderland   = normalize(wonderland_raw)
lookingglass = normalize(lookingglass_raw)

print(f"Wonderland chars: {len(wonderland):,} | Looking-Glass chars: {len(lookingglass):,}")


### Helpers: Tokenization

In [ ]:
WORD_RE = re.compile(r"[A-Za-z']+")  # keep apostrophes in words (e.g., don't -> don't)

def words(text: str):
    """Simple word tokenizer (lowercased, ASCII letters + apostrophes).

    Pros
    ----
    - Very fast and dependency-free.
    - Good enough for frequency/keyness demonstrations.

    Cons
    ----
    - No punctuation words, no sentence boundaries, no POS tags.
    - May treat possessives inconsistently across sources.

    Returns
    -------
    list[str]
        Lowercased word words.
    """
    return WORD_RE.findall(text.lower())


def sentences(text: str):
    """Naive sentence splitter using punctuation boundaries.

    Uses a regex to split on '.', '!', '?' followed by whitespace.
    Because this is heuristic, treat results as approximate.

    Returns
    -------
    list[str]
        Sentence-like strings.
    """
    return [s.strip() for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]



wonderland_words = words(wonderland)
lookingglass_words = words(lookingglass)

wonderland_sentences = sentences(wonderland)
lookingglass_sentences = sentences(lookingglass)

print(f"Wonderland words: {len(wonderland_words):,} | Looking-Glass words: {len(lookingglass_words):,}")
print(f"Wonderland sentences: {len(wonderland_sentences):,} | Looking Glass sentences: {len(lookingglass_sentences):,}")


### Estimate -ly Adverb Rate

In [ ]:

def adverb_rate(words):
    adverbs = [w for w in words if w.endswith('ly') and len(w)>2]
    return len(adverbs), len(words), (len(adverbs)/len(words))*100

a_adv, a_total, a_pct = adverb_rate(wonderland_words)
g_adv, g_total, g_pct = adverb_rate(lookingglass_words)
print(f"Wonderland: {a_adv}/{a_total} = {a_pct:.2f}%")
print(f"Looking Glass: {g_adv}/{g_total} = {g_pct:.2f}%")


**Prompt:** Inspect a sample of detected -ly words. Which are true adverbs vs. adjectives/nouns? How would you refine the rule?

In [ ]:
# Show some adverbs from Wonderland
adverbs_wonderland = [w for w in wonderland_words if w.endswith('ly') and len(w)>2]
print(f"Wonderland adverbs (first 20): {adverbs_wonderland[:20]}")

Let's use a smarter way to find adverbs:

In [ ]:
# Run this cell once
import spacy
from spacy.cli import download

try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    download("en_core_web_sm")          # downloads the small English model
    nlp = spacy.load("en_core_web_sm")  # try again


In [ ]:
def strip_gutenberg_markup(t: str) -> str:
    # remove lone underscores and _italic_ markup
    t = re.sub(r"\b_+\b", " ", t)
    t = re.sub(r"_([A-Za-z]+)_", r"\1", t)
    return t

# 1) Pre-clean, then parse
text_clean = strip_gutenberg_markup(wonderland)  # or your variable
doc = nlp(text_clean)

true_adverbs = [t.text for t in doc if t.pos_ == "ADV"]
true_adverbs[:20]
